# Author Name Recon Part 3

Following part 2, there are three main cases of ambiguity that can be resolved by affiliations and other methods, we do so here using a number of strategies. Given the disjointed nature of the affiliations data, I will also use this process to iteratively consolidate the two sets of affiliations for common 

This part combines

## Three cases of ambiguity:
1. same name, different person
   - is in same time period -> require affiliations
   - not in same time period -> resolve with significant significant publishing year barrier
2. similar name or alias, same person
   - has multiple resolution from no middle name to has middle name: require affiliations
   - spelling error possibilities. Direct correction in data - require research on individual articles for resolution, restricted to this data set only
3. has multiple resolutions from contraction: require affiliations
   - generate list of contractions and associated possibilities if full name exists


Notes
We only care about authors of articles (any not misc or reviews) between 1940 and 2020
We can only process those articles with affiliations recorded or explicitly set to none

In [851]:
import pandas as pd
# from unidecode import unidecode
import re
from datetime import date
import json
import numpy as np
import string
import time
from matplotlib import pyplot
import matplotlib as plt
# from fuzzywuzzy import fuzz 
# from fuzzywuzzy import process 
# set column options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
from functools import reduce
import pprint


In [852]:
base_path="/Users/sijiawu/Work/Thesis/Data/Affiliations/"

In [853]:
with open(base_path+"author_proc_w_aff.json") as f: 
    data = f.read() 

proc_auths_all = json.loads(data) 

In [854]:
content_ex=['MISC',  'Discussion', 'Review', 'Review2',"Errata"]
content=['Article', 'Comment', 'Reply', 'Rejoinder']
jid=["aer","ecta","jpe","qje","res"]

In [855]:
proc_auths_all['https://doi.org/10.1093/qje/qjaa012']

{'authors': {'0': {'raw': 'Enke, Benjamin',
   'init': 'benjamin enke',
   'auth_suffix': [],
   'a1': 'benjamin enke',
   'a2': 'b. enke',
   'a3': 'b. enke',
   'affs': {'1': {'aff_department_final': None,
     'aff_main_final': 'harvard university',
     'aff_subunit_final': None,
     'alt_final': None,
     'manual': 'manual',
     'screened': 'NA',
     'country': 'United States'},
    '2': {'aff_department_final': None,
     'aff_main_final': 'national bureau of economic research - nber',
     'aff_subunit_final': None,
     'alt_final': None,
     'manual': 'manual',
     'screened': 'NA',
     'country': 'United States'}}}},
 'year': 2020,
 'content_type': 'Article',
 'jid': 'qje'}

In [856]:
a1=[]
a2=[]
a3=[]
ini=[]
condense={}
f=[]
d=[]
o=[]
e=[]
k=0
for i in proc_auths_all.keys():
    if (proc_auths_all[i]["content_type"] in content)&(proc_auths_all[i]["year"]>=1940):
            for j in proc_auths_all[i]['authors'].keys():
                temp=proc_auths_all[i]['authors'][j]
                a1.append(temp['a1'])
                a2.append(temp['a2'])
                a3.append(temp['a3'])
                ini.append(temp["init"])
                mains=[]
                if len(temp['affs'])>0:
                    for l in temp["affs"].keys():
                        mains.append(temp["affs"][l]["aff_main_final"])
                proc_auths_all[i]['authors'][j]["affs_min"]=list(set(mains))
                condense[k]={"auth_ord":j,"raw":temp["raw"], "init":temp["init"], "suffix":temp["auth_suffix"],"a1":temp['a1'], "a2": temp["a2"], "a3": temp["a3"], "last": temp['a1'].split(" ")[-1],"affs": set(mains), "year":proc_auths_all[i]["year"],"content_type":proc_auths_all[i]["content_type"],"jid":proc_auths_all[i]["jid"], "url": i}
                k=k+1

In [857]:
proc_auths_all['https://doi.org/10.1093/qje/qjaa012']

{'authors': {'0': {'raw': 'Enke, Benjamin',
   'init': 'benjamin enke',
   'auth_suffix': [],
   'a1': 'benjamin enke',
   'a2': 'b. enke',
   'a3': 'b. enke',
   'affs': {'1': {'aff_department_final': None,
     'aff_main_final': 'harvard university',
     'aff_subunit_final': None,
     'alt_final': None,
     'manual': 'manual',
     'screened': 'NA',
     'country': 'United States'},
    '2': {'aff_department_final': None,
     'aff_main_final': 'national bureau of economic research - nber',
     'aff_subunit_final': None,
     'alt_final': None,
     'manual': 'manual',
     'screened': 'NA',
     'country': 'United States'}},
   'affs_min': ['harvard university',
    'national bureau of economic research - nber']}},
 'year': 2020,
 'content_type': 'Article',
 'jid': 'qje'}

# Approach
1. At a1, problem of are these two authors the same person despite having the same spelling?
2. At a2, are these two the same person despite having different spelling originally?
3. At a3, are these two the same person despite having different spelling originally?
4. At a3, if author names only result in a unique author




In [858]:
proc_auth_flat=pd.DataFrame(condense).transpose()


In [859]:
uni=list(set(a1))
uni2=list(set(a2))
uni3=list(set(a3))
unii=list(set(ini))
proc_auth_flat['a1_order']=None
proc_auth_flat['a2_order']=None
proc_auth_flat['a3_order']=None
proc_auth_flat['init_order']=None
for i in proc_auth_flat.index:
    index = uni.index(proc_auth_flat.loc[i, "a1"])
    index2 = uni2.index(proc_auth_flat.loc[i, "a2"])
    index3 = uni3.index(proc_auth_flat.loc[i, "a3"])
    indexini=unii.index(proc_auth_flat.loc[i, "init"])

    proc_auth_flat.loc[i, 'a1_order']=index
    proc_auth_flat.loc[i, 'a2_order']=index2
    proc_auth_flat.loc[i, 'a3_order']=index3
    proc_auth_flat.loc[i, 'init_order']=indexini

proc_auth_flat['fl']=proc_auth_flat['a1'].str.split(' ').str[0].str.strip() +" " +proc_auth_flat['a1'].str.split(" ").str[-1].str.strip()


In [860]:
proc_auth_flat['a1_tk_count']=proc_auth_flat['a1'].str.split(' ').apply(len)

In [861]:
proc_auth_flat['a1_tk_count'].value_counts()

a1_tk_count
2     24350
3     21357
4       679
5        45
1         2
6         2
10        1
Name: count, dtype: int64

In [862]:
proc_auth_flat.to_pickle(base_path+"proc_auth_aff_flat.pkl")

In [863]:
proc_auth_flat.dtypes

auth_ord        object
raw             object
init            object
suffix          object
a1              object
a2              object
a3              object
last            object
affs            object
year            object
content_type    object
jid             object
url             object
a1_order        object
a2_order        object
a3_order        object
init_order      object
fl              object
a1_tk_count      int64
dtype: object

In [864]:

# a1_r the unique cases all the way from a1 to a3
# a1_s unique cases within a1 and a2 where first names distinguish them out
# a1_t some ambiguity with first name and last name the same but either different or missing middle names
# a1_u if all same token length, we can just check the middle names against each other
# a1_v 

In [865]:
# len(proc_auth_flat[proc_auth_flat['r_a1']==-1]['a1'].unique())


In [866]:
def get_tk_dict(num):
    dict={}
    for i in range(2,num+1,1):
        dict[i]={}
    return dict

def get_a2(name):
    temp=name.split(' ')
    out_name=""
    for i in range(len(temp)-1):
        out_name+=temp[i][0]+ ". "
    return out_name+temp[-1]



In [867]:
proc_auth_flat.shape

(46436, 19)

In [868]:
proc_auth_flat[(proc_auth_flat["a1"]==proc_auth_flat["a2"])&(proc_auth_flat["a2"]!=proc_auth_flat["a3"])].shape

(3297, 19)

In [869]:
proc_auth_flat[(proc_auth_flat["a1"]==proc_auth_flat["a2"])&(proc_auth_flat["a2"]==proc_auth_flat["a3"])].shape

(832, 19)

In [870]:
proc_auth_flat["ini"]=proc_auth_flat["a1"].str[0]
proc_auth_flat["inil"]=proc_auth_flat["a1"].str[0]+". " + proc_auth_flat["last"]

In [871]:
len(proc_auth_flat[(proc_auth_flat["a1"]==proc_auth_flat["a2"])]["a2"].unique())

1908

In [872]:
proc_auth_flat.columns

Index(['auth_ord', 'raw', 'init', 'suffix', 'a1', 'a2', 'a3', 'last', 'affs',
       'year', 'content_type', 'jid', 'url', 'a1_order', 'a2_order',
       'a3_order', 'init_order', 'fl', 'a1_tk_count', 'ini', 'inil'],
      dtype='object')

In [873]:
ln=list(proc_auth_flat[(proc_auth_flat["a1"]!=proc_auth_flat["a2"])]["last"])
fini=list(proc_auth_flat[(proc_auth_flat["a1"]!=proc_auth_flat["a2"])]["ini"])
finil=list(proc_auth_flat[(proc_auth_flat["a1"]!=proc_auth_flat["a2"])]["inil"])


In [874]:
auth_is=proc_auth_flat[(proc_auth_flat["a1"]==proc_auth_flat["a2"])&(proc_auth_flat["inil"].isin(finil))][["year","jid"]].value_counts().reset_index()
# auth_is.sort_values(by="year")

In [875]:
# let's for now ignore the fact two entries of anderson a. cooper at the a1 stage may be different people
# The first goal is to get this to a point that we can say a2 and a3 name variations no longer exist in a1
# a2 and a3 variations do not inform on a1 variations
# The trick is to start at the point that the names themselves no longer offer any information as to whether two entries are the same author

# Step 1: separate out all unique a1 names and find the a1 names that need to be matched against that unique a1 set
# use only inherent name properties, no affiliations yet


# strat: assuming if you were to resolve a2 and a3 to a1 format names
# best case?:
# jaegen aasness has a1 of jaegen aasness and a2 of j. aasness and a3 of j. aasness and importantly there are no variations in a1 of jaegen aasness
# if jaegen aasness is the only j. aasness that means if we were to group all the a3 names that match j. aasness 
# also accounts for the case if j. aasness is already at a1 and no a2 or a3 ambiguity exists.
# they will only have one id of jaegen aasness at a1 with and his affiliations will all intersect. 
# These just have the issue that they may be same name, different people at the a1 stage. so lets record their a1 names and orders

# let's take the better case no a2 and a3 formats in a1 - no one decided to contract their name:
# tore olsen and trond e. olsen at a1 resolve to t. olsen at a2 and t. e. olsen at a2, and t. olsen at a3 respectively.
# this will account for the andrew cain and anders cain case as well
# their first names are sufficiently different to tell them apart that they are different people. but their full first name needs to be present in the a1 format first.
# ie: given the same a3 in a set, if a3 or a2 variations do not occur in a1's of this set and all first names are in full then two differing a1's are distinct.
# now, in this set we will have for example - henry j. aaron, henry aaron and harry aaron and harry k. aaron and harry a. aaron and hamlin aaron:
    # group harry, henry and hamlin separately because they are distinct
    # within each group:
        # hamlin is fine and we assign him off
        # harry's - harry k. aaron and harry a. aaron are distinct and can be assigned if you check the middle name contractions, 
        # however, harry aaron is unassignable so hold to be matched for later - include the a1 potential match ids
        # similarly, henry aaron needs to be distinguished from henry j. aaron so hold to be matched for later with the associated potential match id

# the worst case scenario
# suppose henry j. aaron, h. j. aaron, h. aaron and henry k. aaron and harry aaron all exist and let's add henry j. k. aaron
# the only thing we know for certain is that henry j. k. aaron is unique at a1
# henry j. aaron and henry k. aaron could be a variation of it or a different person
# harry aaron could be h. j. aaron
# h. j. aaron is likely a variation of henry j. aaron or henry j. aaron
# h. aaron is likely a variation of henry j. aaron h. j. aaron henry k. aaron harry aaron

proc_auth_flat["r_a1"]=-1
proc_auth_flat["r_a1_name"]=""
proc_auth_flat["p_a1"]=pd.Series(dtype='object')
proc_auth_flat["p_a1_name"]=""
c={
    'a1_r':0,
    'a1_s':0,
    'a1_t':0,
    'a1_u':0,
    'a1_v':0,
    'a1_x':0
}
manuals=0
leftovers=0
for i in range(len(uni3)):
    # print(uni3[i])
    a3_set=proc_auth_flat[proc_auth_flat['a3_order']==i]  # get all rows of of dataframe with the same a3 at i eg: w. leontief
    a1_set=list(set(a3_set['a1_order'])) # ax_order is the id for ax unique names
    a1_n_set=set(a3_set['a1']) # get the set of a1 variations of w. leontief
    a2_n_set=set(a3_set['a2']) # get the set of a2 variations of w. leontief
    a3_n_set=set(a3_set['a3']) # get the set of a3 variations of w. leontief
    
    if len(a1_set)==1:
        # print(a1_set[0])
        proc_auth_flat.loc[proc_auth_flat['a3_order']==i, 'r_a1']=a1_set[0]
        proc_auth_flat.loc[proc_auth_flat['a3_order']==i, 'r_a1_name']=list(a1_n_set)[0]
        c['a1_r']=c['a1_r']+1
    else:
        fl_set=set(a3_set['fl'])
        # there is no first name contrations to match only middle name contractions
        if (len(a1_n_set.intersection(a2_n_set))==0) & (len(a1_n_set.intersection(a3_n_set))==0):
            # print(a1_n_set)
            # print(a1_n_set)
            # print(fl_set)
            for j in fl_set:
                fl_subset=proc_auth_flat[proc_auth_flat['fl']==j]
                fl_set_red=set(fl_subset['a1'])
                if len(fl_set_red)==1:
                    proc_auth_flat.loc[proc_auth_flat['fl']==j, 'r_a1']=list(set(fl_subset['a1_order']))[0]
                    proc_auth_flat.loc[proc_auth_flat['fl']==j, 'r_a1_name']=list(set(fl_subset['a1']))[0]
                    c['a1_s']=c['a1_s']+1                    
                else:
                    # print(j)
                    tk_set=set(fl_subset['a1_tk_count'])
                    if len(tk_set)==1:
                        for k in fl_set_red:
                            # print(k)
                            proc_auth_flat.loc[proc_auth_flat['a1']==k, 'r_a1']=list(set(proc_auth_flat[proc_auth_flat['a1']==k]['a1_order']))[0]
                            proc_auth_flat.loc[proc_auth_flat['a1']==k, 'r_a1_name']=list(set(proc_auth_flat[proc_auth_flat['a1']==k]['a1']))[0]
                        c['a1_u']=c['a1_u']+1
                    else:
                        tk_org=get_tk_dict(max(tk_set))
                        tk_all={}
                        tk_set_l=list(tk_set)
                        tk_set_l.sort(reverse=True)
                        max_tk=max(tk_set_l)
                        min_tk=min(tk_set_l)

                        for k in fl_set_red:
                            tk_org[len(k.split(' '))][k[len(k.split(' ')[0]):-len(k.split(' ')[-1])].strip()]=k
                            tk_all[k[len(k.split(' ')[0]):-len(k.split(' ')[-1])].strip()]=k
                        
                        mx=[]
                        # if anything upwards has 
                        for k in tk_set_l:
                            # print(k)
                            for l in tk_org[k]:
                                pot=[]
                                for m in mx:
                                    if l in m:
                                        pot.append(m)
                                if len(pot)==0:
                                    mx.append(l)
                                    # print(tk_org[k][l])
                                    proc_auth_flat.loc[proc_auth_flat['a1']==tk_org[k][l], 'r_a1']=list(set(proc_auth_flat[proc_auth_flat['a1']==tk_org[k][l]]['a1_order']))[0]
                                    proc_auth_flat.loc[proc_auth_flat['a1']==tk_org[k][l], 'r_a1_name']=list(set(proc_auth_flat[proc_auth_flat['a1']==tk_org[k][l]]['a1']))[0]
                                else:
                                    for n in range(len(pot)):
                                        pot[n]=tk_all[pot[n]]
                                    # print(tk_org[k][l])
                                    # print(pot)
                                    mask=proc_auth_flat['a1']==tk_org[k][l]
                                    # print(np.array(pot))
                                    proc_auth_flat.loc[mask, 'p_a1']=json.dumps(pot)
                        c['a1_v']=c['a1_v']+1
        # groups with first name contracted and first name not contracted    
        # Separate first name contracted from first name not contracted    
        else:
            # print()
            # print("a1 set")
            # print(a1_n_set)

            a_a3=a1_n_set.intersection(a3_n_set)
            a_a2=a1_n_set.intersection(a2_n_set)

            m_a1=a1_n_set.intersection(a3_n_set)|a1_n_set.intersection(a2_n_set)
            a_a1=a1_n_set^m_a1

            tk_set=set(fl_subset['a1_tk_count'])

            # for j in a_a1:
            #     if len(j.split(" "))>2:
            #         print(j+"**")
            #     else:
            #         print(j+"*")
            
            a_a1_set=a3_set[a3_set["a1"].isin(list(a_a1))] # all names that are only a1 has middle name contracted and no middle name
            # fl_subset=proc_auth_flat[proc_auth_flat['fl']==j]

            # display(a_a1_set)
            fl_set=set(a_a1_set['fl'])

            # if len(m_a1)>0:
            #     print("a1 intersect with a3")
            #     print(a_a3)
            #     print("a1 intersect with a2")
            #     print(a_a2)
            #     print("a2 and a3 in a1")
            #     print(m_a1)
            #     print("names with full first names")
            #     print(a_a1)
            #     print("fl set")
            #     print(fl_set)

            # print()
            for j in list(fl_set):
                fl_subset=proc_auth_flat[proc_auth_flat['fl']==j] # we look at each first and last name combo. assuming the first name is not contracted
                # display(fl_subset)
                fl_set_red=set(fl_subset['a1'])
                # print(fl_set_red)
                if len(fl_set_red)==1: #if the first name is unique such that there is no middle contraction that confuses the two then it is unique at a1.
                    # print("execute")
                    proc_auth_flat.loc[proc_auth_flat['fl']==j, 'r_a1']=list(set(fl_subset['a1_order']))[0]
                    proc_auth_flat.loc[proc_auth_flat['fl']==j, 'r_a1_name']=list(set(fl_subset['a1']))[0]
                    # print(list(set(fl_subset['a1_order']))[0])
                    c['a1_s']=c['a1_s']+1                    
                else: #let's resolve cases that have middle names contracted and not and matching first names and last names: john d. green and john green and john m. green
                    # print("resolving first last cases with middle names")
                    tk_set=set(a_a1_set['a1_tk_count']) # set of count of tokens associated with this set eg: 2 for john green, 3 for john d. green and john m. green
                    tk_org=get_tk_dict(max(tk_set)) # get a dictionary prepopulated with keys descending from the highest token count eg: 3
                    tk_all={}    # set up a holder for key value where key is the middle value
                    tk_set_l=list(tk_set) # get a list of the token count numbers, this is a set lets say 4 3 5 2, remember it's always greater than two
                    tk_set_l.sort(reverse=True) # reverse sort the list from largest to smallest for john greens this will be 3, 2
                    max_tk=max(tk_set_l) # most tokens = 3 in the set
                    min_tk=min(tk_set_l) # min tokens = 2 in the set

                    for k in fl_set_red:
                        mid=k[len(k.split(' ')[0]):-len(k.split(' ')[-1])].strip() # get the middle name contractions 
                        mid_l=len(k.split(' ')) # get number of tokens in the name
                        tk_org[mid_l][mid]=k # populate the dictionary that allocates by token count then contraction
                        tk_all[mid]=k # populate the dictionary that allocates by contraction since this are unique variations of john x green. x will be unique and there will be no hashed conflict
                        # print(tk_org)
                    # print(tk_org)
                    # print(tk_all)

                    mx=[]
                                            # if anything upwards has 
                    for k in tk_set_l:  # note that tk_set_l is a token count from longest to shortest in this case [3, 2] but of all john x greens where x can be of [3, 2] 
                        # print(k)
                        for l in tk_org[k]: # get the dictionary of all values with token lengths of size k, we start with 3 nothing above 2 below
                            pot=[]  # this is a list of potentials middle name contractions that l is a subset of contractions of names with longer token counts 
                            # print("id: "+l) # we have john green, john r. green, john g. green. since the longest token length is 3 , the first candidate is "r." or "g." 
                            for m in mx:  # there are no higher order middle name contractions for r. or g. so we are iterating through an empty mx,  
                                if l in m: # at k=2 the candidate is "" and assuming we have appended "r." and "g." to mx john green will be marked as having r. and g. as potential matches
                                    pot.append(m) # this is because "" is a subset of "r." and "g." similarly "r." would be a subset or "l. r."

                            if len(pot)==0: # if the list of potential higher order matches is empty then we consider this name completely unique at a1
                                mx.append(l) # append the middle contraction to mx and assign all occurences with the a1_order
                                # print(tk_org[k][l])
                                proc_auth_flat.loc[proc_auth_flat['a1']==tk_org[k][l], 'r_a1']=list(set(proc_auth_flat[proc_auth_flat['a1']==tk_org[k][l]]['a1_order']))[0]
                                proc_auth_flat.loc[proc_auth_flat['a1']==tk_org[k][l], 'r_a1_name']=list(set(proc_auth_flat[proc_auth_flat['a1']==tk_org[k][l]]['a1']))[0]

                            else: 
                                for n in range(len(pot)): # in pot, I want replace each middle name contraction with the full a1 name.
                                    pot[n]=tk_all[pot[n]]
                                # print(tk_org[k][l])
                                # print(pot)
                                # print(np.array(pot))
                                proc_auth_flat.loc[proc_auth_flat['a1']==tk_org[k][l], 'p_a1']=json.dumps(pot) # set the a1 name potentials for matches of full name
                    # print(mx)
                    c['a1_v']=c['a1_v']+1


            # contains cases of contracted first and middle names: ie: all the variants of w. leontief  
            # let's say w. leontief, w. w. leontief, wassily w. leontief, wassily leontief, wassily r. leontief, wendell leontief, wendell w. leontief exist at a1
            # and we must allocate w. leontief and w. w. leontief.
            # assume that we don't have exact same full name middle name who are the same people yet.
            # we need a to structure the above into two dictionaries like so where the numbers are how many tokens there are in the author name if split by " ":
            # notice that the smallest unit of the dictionary is an array. This is unlike the previous case of matching full first names with no middle name which we knew for certain that 
            # a1s will full first names= f_a1_dict {
            #  2: "": ["wassily leontief","wendell leontief"],
            #  3: "w.":["wendell w. leontief", "wassily w. leontief"], 
            #     "r.": ["wassily r. leontief"]
            #}
            # a1s with contracted first names=c_a1_dict {
            #  2: "": ["w. leontief"],
            #  3: "w.":["w. w. leontief"],
            #}
            # I want to avoid circular referencing, as a rule, this means only something equally or more complex is a match in each case
            # Given w. leontief: it potentially matches everything in length we ignore 2+ f_a1_dict and anything length 3+ in c_a1_dict
            # Given w. w. leontief: it potentially matches f_a1_dict[3]["w"] or 3+ in f_a1_dict and 4+ if that exists in c_a1_dict

            # since contractions are always matched into non-contracted first names this means 
            # Given w. w. leontief, wasilly leontief is also a match, given w. r. k. leontief: wassily r. leontief, and wassily leontief is a potential match
            
            f_a1=proc_auth_flat[proc_auth_flat["a1"].isin(a_a1)]
            f_a1_tk_set=set(f_a1['a1_tk_count'])

            if len(a_a1)==0:
                f_a1_dict=get_tk_dict(0)
            else:
                f_a1_dict=get_tk_dict(max(f_a1_tk_set))

            f_a1_set_red=set(f_a1['a1'])
            f_tk_all={}

            for k in f_a1_set_red:
                # print(k)
                mid=k[len(k.split(' ')[0]):-len(k.split(' ')[-1])].strip()
                m_l=len(k.split(' ')) #length of the name ie: tokens
                if mid in f_a1_dict.keys():
                    f_a1_dict[m_l][mid].append(k)
                    f_tk_all[mid].append(k)
                else:
                    f_a1_dict[m_l][mid]=[k]
                    f_tk_all[mid]=[k]

            # print(f_a1_dict)
            # print(f_tk_all)

            c_a1=proc_auth_flat[proc_auth_flat["a1"].isin(m_a1)]
            c_a1_tk_set=set(c_a1['a1_tk_count'])
            if len(m_a1)==0:
                c_a1_dict=get_tk_dict(0)
            else:
                c_a1_dict=get_tk_dict(max(c_a1_tk_set))
            c_a1_set_red=set(c_a1['a1'])
            c_tk_all={}
            

            for k in c_a1_set_red:
                # print(k)
                mid=k[len(k.split(' ')[0]):-len(k.split(' ')[-1])].strip()
                m_l=len(k.split(' ')) #length of the name ie: tokens
                if mid in c_a1_dict.keys():
                    c_a1_dict[m_l][mid].append(k)
                    c_tk_all[mid].append(k)
                else:
                    c_a1_dict[m_l][mid]=[k]
                    c_tk_all[mid]=[k]
                
            # print(c_a1_dict)
            # print(c_tk_all)

            

            for k in m_a1:
                mx=[]
                mid=k[len(k.split(' ')[0]):-len(k.split(' ')[-1])].strip()
                m_l=len(k.split(" "))
                # print(k + " " + str(m_l))
                # print("candidates")
                leftovers+=1
                pot=[]
                # let's say m_l is 3 and mid is "x." then we want everything >=3 in f_a1_dict
                # everthing >=4 in c_a1_dict where mid is a subset
                # everything < 3 where "x." is superset
                # make the pot - potential a1 match candidates
                for l in f_a1_dict.keys():
                    if l>=m_l:
                        for n in f_a1_dict[l]:
                            if mid in n:
                                for p in f_a1_dict[l][n]:
                                    pot.append(p)

                for l in c_a1_dict.keys():
                    if l>=(m_l+1):
                        for n in c_a1_dict[l]:
                            if mid in n:
                                for p in c_a1_dict[l][n]:
                                    pot.append(p)
                
                for l in f_a1_dict.keys():
                    if l<m_l:
                        for n in f_a1_dict[l]:
                            if n in mid:
                                for p in f_a1_dict[l][n]:
                                    pot.append(p)
                if len(pot)==0:
                    proc_auth_flat.loc[proc_auth_flat['a1']==k, 'r_a1']=list(set(proc_auth_flat[proc_auth_flat['a1']==k]['a1_order']))[0]
                    proc_auth_flat.loc[proc_auth_flat['a1']==k, 'r_a1_name']=list(set(proc_auth_flat[proc_auth_flat['a1']==k]['a1']))[0]

                else:
                    # print(pot)
                    proc_auth_flat.loc[proc_auth_flat['a1']==k, 'p_a1']=json.dumps(pot)
            manuals+=1
            # print()

    # if i >300:
    #     break


In [876]:
manuals

860

In [877]:
proc_auth_flat.head()

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,a3_order,init_order,fl,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name
0,0,"Enke, Benjamin",benjamin enke,[],benjamin enke,b. enke,b. enke,enke,"{harvard university, national bureau of economic research - nber}",2020,...,6730,6653,benjamin enke,2,b,b. enke,6588,benjamin enke,NaN,
1,0,Lauren Falcao Bergquist,lauren falcao bergquist,[],lauren f. bergquist,l. f. bergquist,l. bergquist,bergquist,{university of michigan},2020,...,9770,3287,lauren bergquist,3,l,l. bergquist,13730,lauren f. bergquist,NaN,
2,1,Michael Dinerstein,michael dinerstein,[],michael dinerstein,m. dinerstein,m. dinerstein,dinerstein,{university of chicago},2020,...,11382,11900,michael dinerstein,2,m,m. dinerstein,11766,michael dinerstein,NaN,
3,0,Dominic Coey,dominic coey,[],dominic coey,d. coey,d. coey,coey,{facebook},2020,...,336,7852,dominic coey,2,d,d. coey,7759,dominic coey,NaN,
4,1,Bradley J. Larsen,bradley j. larsen,[],bradley j. larsen,b. j. larsen,b. larsen,larsen,"{national bureau of economic research - nber, stanford university}",2020,...,12387,8545,bradley larsen,3,b,b. larsen,8447,bradley j. larsen,NaN,


In [878]:
proc_auth_flat[proc_auth_flat["p_a1"].isna()==False].head()

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,a3_order,init_order,fl,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name
14,4,Heidi Williams,heidi williams,[],heidi williams,h. williams,h. williams,williams,"{national bureau of economic research - nber, stanford university}",2020,...,4212,11149,heidi williams,2,h,h. williams,-1,,"[""heidi l. williams""]",
20,1,John Haltiwanger,john haltiwanger,[],john haltiwanger,j. haltiwanger,j. haltiwanger,haltiwanger,"{national bureau of economic research - nber, university of maryland - college park}",2020,...,2316,412,john haltiwanger,2,j,j. haltiwanger,-1,,"[""john c. haltiwanger""]",
64,0,David Autor,david autor,[],david autor,d. autor,d. autor,autor,"{massachusetts institute of technology - mit, national bureau of economic research - nber}",2020,...,4567,17399,david autor,2,d,d. autor,-1,,"[""david h. autor""]",
66,2,Gordon Hanson,gordon hanson,[],gordon hanson,g. hanson,g. hanson,hanson,"{harvard university, national bureau of economic research - nber}",2020,...,8058,3494,gordon hanson,2,g,g. hanson,-1,,"[""gordon h. hanson""]",
120,1,Laura Veldkamp,laura veldkamp,[],laura veldkamp,l. veldkamp,l. veldkamp,veldkamp,"{national bureau of economic research - nber, columbia university, london school of economics and political science}",2020,...,3809,420,laura veldkamp,2,l,l. veldkamp,-1,,"[""laura l. veldkamp""]",


In [879]:
proc_auth_flat[proc_auth_flat["r_a1_name"]!=""].shape

(41560, 25)

In [880]:
proc_auth_flat.head()

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,a3_order,init_order,fl,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name
0,0,"Enke, Benjamin",benjamin enke,[],benjamin enke,b. enke,b. enke,enke,"{harvard university, national bureau of economic research - nber}",2020,...,6730,6653,benjamin enke,2,b,b. enke,6588,benjamin enke,NaN,
1,0,Lauren Falcao Bergquist,lauren falcao bergquist,[],lauren f. bergquist,l. f. bergquist,l. bergquist,bergquist,{university of michigan},2020,...,9770,3287,lauren bergquist,3,l,l. bergquist,13730,lauren f. bergquist,NaN,
2,1,Michael Dinerstein,michael dinerstein,[],michael dinerstein,m. dinerstein,m. dinerstein,dinerstein,{university of chicago},2020,...,11382,11900,michael dinerstein,2,m,m. dinerstein,11766,michael dinerstein,NaN,
3,0,Dominic Coey,dominic coey,[],dominic coey,d. coey,d. coey,coey,{facebook},2020,...,336,7852,dominic coey,2,d,d. coey,7759,dominic coey,NaN,
4,1,Bradley J. Larsen,bradley j. larsen,[],bradley j. larsen,b. j. larsen,b. larsen,larsen,"{national bureau of economic research - nber, stanford university}",2020,...,12387,8545,bradley larsen,3,b,b. larsen,8447,bradley j. larsen,NaN,


In [881]:
len(proc_auth_flat["a1"].unique())

17304

In [882]:
leftovers

911

In [883]:
proc_auth_flat["content_type"].value_counts()

content_type
Article      43431
Comment       1807
Reply         1064
Rejoinder      134
Name: count, dtype: int64

In [884]:
len(proc_auth_flat[proc_auth_flat["init"]==proc_auth_flat["a1"]]["a1"].unique())

16238

In [885]:
len(proc_auth_flat[proc_auth_flat["init"]!=proc_auth_flat["a1"]]["init"].unique())

1233

In [886]:
proc_auth_flat[proc_auth_flat["r_a1"]==-1].shape

(4876, 25)

There are 4876 author names that we need to resolve. They either are aliases of authors who have a more complex name listed or they are authors who share the same name or alias as another author. We only resolve to a1 for ease of distinguishing between authors. If it is found that there are two authors of the same name, then we use the initial names or aliases to distinguish the two.

In [887]:
# load in recon list
with open('authors_recon.json') as f:
    a1_recon= json.load(f)

# compile list of author names that this list resolves
parsed_lst=[]

for i in a1_recon:
    i["kauthor"]
    parsed_lst.append(i["cinfo"][0])

combined = [item for sublist in parsed_lst for item in sublist]

In [888]:
def check_name_resolution(name, url):
    flag=False
    temp=[]
    for p in a1_recon:
        if name in p["cinfo"][0]:
            if name == p["kauthor"]:
                flag=True
                temp.append(p)
            else:
                temp.append(p)
    if len(temp)==1:
        return [flag, temp, 1]
    elif len(temp)>1:
        return [flag, temp, len(temp)]
    else:
        return [flag,"",0]

    

In [889]:
for i in a1_recon:
   pots=check_name_resolution(i["kauthor"],"")
   if (pots[0]==True)&(pots[2]>1):
      print(pots) # the resolution

   # for j in i["cinfo"]:
   #    print(j)
      print()



# tuple[0]: list of a1 keys to resolve
# tuple[1]: correct name of author
# tuple[2]: supporting doc that confirms resolution
# tuple[3]: resolution categories and author cats
# tuple[4]: notes - perhaps on variations eg: ae converted from ø
# tuple[5]: set of urls to resolve if a1 key has conflict

[True, [{'kauthor': 'christopher j. harris', 'cinfo': [['christopher harris', 'christopher j. harris'], '', ['aff chain'], '', ['https://www.jstor.org/stable/1911658', 'https://www.jstor.org/stable/2297616', 'https://www.jstor.org/stable/2297442', 'https://www.jstor.org/stable/2297825', 'https://www.jstor.org/stable/2298124', 'https://www.jstor.org/stable/2117834', 'https://www.jstor.org/stable/2171906', 'https://www.jstor.org/stable/2138887', 'https://www.jstor.org/stable/2999588', 'https://www.jstor.org/stable/2692249', 'https://www.jstor.org/stable/2695893', 'https://www.jstor.org/stable/26372497']]}, {'kauthor': 'christopher harris', 'cinfo': [['christopher harris'], '', ['aff chain'], '', ['https://www.jstor.org/stable/42920977']]}], 2]

[True, [{'kauthor': 'john h. boyd, iii', 'cinfo': [['john h. boyd', 'john h. boyd, iii'], 'https://john-boyd.com/vita.html', [], '', ['https://www.jstor.org/stable/2938264']]}, {'kauthor': 'john h. boyd', 'cinfo': [['john h. boyd'], 'https://asset

In [890]:
proc_auth_flat.columns

Index(['auth_ord', 'raw', 'init', 'suffix', 'a1', 'a2', 'a3', 'last', 'affs',
       'year', 'content_type', 'jid', 'url', 'a1_order', 'a2_order',
       'a3_order', 'init_order', 'fl', 'a1_tk_count', 'ini', 'inil', 'r_a1',
       'r_a1_name', 'p_a1', 'p_a1_name'],
      dtype='object')

In [891]:
# Note: this is not robust code. It requires that if there is a URL resolution, it is such that the entry has no other information to implement ito author name recon

for i in a1_recon:
    resolution=i["kauthor"]
    l=len(i["cinfo"])
    urls=[]
    if l==5:
        print(resolution)
        urls=i["cinfo"][4]
        pprint.pprint(i)
        
    if len(urls)>0:
        for j in urls:
            proc_auth_flat.loc[proc_auth_flat["url"]==j,"p_a1_name"]=resolution
    else:
        for j in i["cinfo"][0]:
            proc_auth_flat.loc[proc_auth_flat["a1"]==j,"p_a1_name"]=resolution
            # print(j)

david yang
{'cinfo': [['david yang'],
           '',
           ['unique'],
           'different from david y. yang',
           ['https://www.jstor.org/stable/25592438']],
 'kauthor': 'david yang'}
john y. campbell
{'cinfo': [['john campbell', 'john y. campbell'],
           'https://scholar.harvard.edu/sites/scholar.harvard.edu/files/campbell/files/campcv-1.pdf',
           ['goog'],
           'confirmed',
           ['https://www.jstor.org/stable/2297552']],
 'kauthor': 'john y. campbell'}
david k. levine
{'cinfo': [['david k. levine', 'david levine'],
           '',
           ['aff chain'],
           '',
           ['https://www.jstor.org/stable/2951505',
            'https://www.jstor.org/stable/10.1086/339717',
            'https://www.jstor.org/stable/3083403',
            'https://www.jstor.org/stable/1884378']],
 'kauthor': 'david k. levine'}
george w. evans
{'cinfo': [['george w. evans', 'george evans'],
           '',
           ['aff chain'],
           'duplicate with 

In [892]:
# proc_auth_flat[proc_auth_flat["a3"]=="m. khan"]
# proc_auth_flat[proc_auth_flat["a3"]=="j. nash"]

In [893]:
parsed=[]
ignore=[]
goog_n=[]
goog=[]
inter=[]
for i in proc_auth_flat[proc_auth_flat["p_a1"].isna()==False].sort_values(by="year").index:
    a1_fn=proc_auth_flat.loc[i, "a1"]
    a1_fn_ord=proc_auth_flat.loc[i, "a1_order"]
    a1_only=proc_auth_flat[proc_auth_flat["a1"]==a1_fn].sort_values(by="year")

    if (a1_fn in combined) or (a1_fn in parsed):
        continue
    else:
        parsed.append(a1_fn)
        
    candidates=json.loads(proc_auth_flat.loc[i, "p_a1"])
    cand_all=proc_auth_flat[proc_auth_flat["a1"].isin(candidates)].sort_values(by="year")
    a1set=reduce(set.union, a1_only['affs'])
    candset=reduce(set.union, cand_all['affs'])
    print("prospect --> candidates")
    print(a1_fn +" - "+str(a1_fn_ord) +" : "+ str(candidates))
    print(a1set)
    # print(candset)
    # print(list(a1_only["year"]))
    # print(list(cand_all["year"]))
    sorted_df = a1_only[["year","url", "affs","a1"]].sort_values(by='year')
    sorted_df["info"]="a"

    result_list = sorted_df.values.tolist()

    sorted_df_c = cand_all[["year","url", "affs","a1"]].sort_values(by='year')
    sorted_df_c["info"]="c"
    result_list_c = sorted_df_c.values.tolist()
        
    if reduce(set.union, cand_all['affs'])==reduce(set.union, a1_only['affs']):
        goog_n.append(a1_fn)
    else:
        inter.append(a1_fn)
    
    print(result_list)
    print(result_list_c)

    print("\n######### SINGLES #########")
    for j in sorted_df_c["a1"].unique():
        print(j)
        combo_df=pd.concat([sorted_df,sorted_df_c[sorted_df_c["a1"]==j]]).sort_values(by='year')
        result_combo=combo_df.values.tolist()
        year_diffs = [result_combo[i+1][0] - result_combo[i][0] for i in range(len(result_combo)-1)]
        if max(year_diffs)>5:
            print("######### FLAG #########")
        pprint.pp(combo_df[["year","url", "affs","info"]].sort_values(by='year').values.tolist())
    if len(sorted_df_c["a1"].unique())>1:
        print("######### MULTI CANDIDATE FLAG #########")
        combo_df=pd.concat([sorted_df,sorted_df_c]).sort_values(by='year')
        result_combo=combo_df.values.tolist()
        year_diffs = [result_combo[i+1][0] - result_combo[i][0] for i in range(len(result_combo)-1)]
        if max(year_diffs)>5:
            print("######### FLAG #########")
        pprint.pp(combo_df[["year","url", "affs","info"]].sort_values(by='year').values.tolist())

    un=cand_all[["a1", "a1_order"]].value_counts().reset_index()

    for j in un.index:
        print("{"+str(un.loc[j,"a1_order"])+":(["+str(a1_fn_ord)+"],\""+str(un.loc[j,"a1"])+"\",\"\",\"\")},")
    print("{"+str(a1_fn_ord)+":(["+str(a1_fn_ord)+"],\""+str(a1_fn)+"\",\"\",\"\")},")
    print("\n\n")
    goog.append(a1_fn)

    for j in candidates:
        print("https://www.google.com/search?q="+j.replace(" ","+"))


In [894]:
len(set(goog)) #if this goes to zero there's nothing to resolve.

0

In [895]:
len(set(inter))

0

In [896]:
len(set(goog_n))

0

In [897]:
len(set(proc_auth_flat[proc_auth_flat["p_a1"].isna()==False]["a1"]))

1780

In [898]:
# can use lambda function here
proc_auth_flat["resolution"]=None
for i in proc_auth_flat.index:
    r=proc_auth_flat.loc[i,"r_a1_name"]
    p=proc_auth_flat.loc[i,"p_a1_name"]
    if r=="":
        proc_auth_flat.loc[i,"resolution"]=p
    else:
        if (p!="") & (r!=""):
            proc_auth_flat.loc[i,"resolution"]=p
        else:
            proc_auth_flat.loc[i,"resolution"]=r

In [899]:
proc_auth_flat[proc_auth_flat["resolution"]==""][["url","p_a1","a1","init","affs","r_a1_name","p_a1_name"]]

,url,p_a1,a1,init,affs,r_a1_name,p_a1_name


In [900]:
proc_auth_flat[proc_auth_flat["p_a1"].isna()==False].head()

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,init_order,fl,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution
14,4,Heidi Williams,heidi williams,[],heidi williams,h. williams,h. williams,williams,"{national bureau of economic research - nber, stanford university}",2020,...,11149,heidi williams,2,h,h. williams,-1,,"[""heidi l. williams""]",heidi l. williams,heidi l. williams
20,1,John Haltiwanger,john haltiwanger,[],john haltiwanger,j. haltiwanger,j. haltiwanger,haltiwanger,"{national bureau of economic research - nber, university of maryland - college park}",2020,...,412,john haltiwanger,2,j,j. haltiwanger,-1,,"[""john c. haltiwanger""]",john c. haltiwanger,john c. haltiwanger
64,0,David Autor,david autor,[],david autor,d. autor,d. autor,autor,"{massachusetts institute of technology - mit, national bureau of economic research - nber}",2020,...,17399,david autor,2,d,d. autor,-1,,"[""david h. autor""]",david h. autor,david h. autor
66,2,Gordon Hanson,gordon hanson,[],gordon hanson,g. hanson,g. hanson,hanson,"{harvard university, national bureau of economic research - nber}",2020,...,3494,gordon hanson,2,g,g. hanson,-1,,"[""gordon h. hanson""]",gordon h. hanson,gordon h. hanson
120,1,Laura Veldkamp,laura veldkamp,[],laura veldkamp,l. veldkamp,l. veldkamp,veldkamp,"{national bureau of economic research - nber, columbia university, london school of economics and political science}",2020,...,420,laura veldkamp,2,l,l. veldkamp,-1,,"[""laura l. veldkamp""]",laura l. veldkamp,laura l. veldkamp


In [901]:
proc_auth_flat[proc_auth_flat["r_a1"]!=-1].head()

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,init_order,fl,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution
0,0,"Enke, Benjamin",benjamin enke,[],benjamin enke,b. enke,b. enke,enke,"{harvard university, national bureau of economic research - nber}",2020,...,6653,benjamin enke,2,b,b. enke,6588,benjamin enke,NaN,,benjamin enke
1,0,Lauren Falcao Bergquist,lauren falcao bergquist,[],lauren f. bergquist,l. f. bergquist,l. bergquist,bergquist,{university of michigan},2020,...,3287,lauren bergquist,3,l,l. bergquist,13730,lauren f. bergquist,NaN,,lauren f. bergquist
2,1,Michael Dinerstein,michael dinerstein,[],michael dinerstein,m. dinerstein,m. dinerstein,dinerstein,{university of chicago},2020,...,11900,michael dinerstein,2,m,m. dinerstein,11766,michael dinerstein,NaN,,michael dinerstein
3,0,Dominic Coey,dominic coey,[],dominic coey,d. coey,d. coey,coey,{facebook},2020,...,7852,dominic coey,2,d,d. coey,7759,dominic coey,NaN,,dominic coey
4,1,Bradley J. Larsen,bradley j. larsen,[],bradley j. larsen,b. j. larsen,b. larsen,larsen,"{national bureau of economic research - nber, stanford university}",2020,...,8545,bradley larsen,3,b,b. larsen,8447,bradley j. larsen,NaN,bradley j. larsen,bradley j. larsen


In [906]:
for i in proc_auth_flat.index:
    a=proc_auth_flat.loc[i,"url"]
    b=proc_auth_flat.loc[i,"auth_ord"]
    c=proc_auth_flat.loc[i, 'resolution']
    proc_auths_all[a]["authors"][b]["resolution"]=c

In [908]:
proc_auth_flat.sort_values(by=["resolution","a3","a2","a1","init","year"])
proc_auth_flat.to_excel(base_path+"auth_proc_w_aff_flat.xlsx", index=False)
proc_auth_flat.to_pickle(base_path+"auth_proc_w_aff_flat.pkl")

with open(base_path+"author_proc_all_w_red.json", "w") as outfile: 
    json.dump(proc_auths_all, outfile, indent=4, default=int)